In [1]:
import os
from groq import Groq
from pathlib import Path
import json
from openai import OpenAI

In [ ]:
# API Keys
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

# Model configurations
LLAMA_MODEL = "llama-3.3-70b-versatile"
DEEPSEEK_MODEL = "deepseek-chat"

# Prompts file
PROMPTS_FILE = "../prompts/l4_07_critical_infra.json"

# Initialize clients
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])
deepseek_client = OpenAI(
    api_key=os.environ["DEEPSEEK_API_KEY"],
    base_url="https://api.deepseek.com"
)

print("✓ API clients initialized")

✓ API clients initialized


In [3]:
def call_llama(prompt: str) -> str:
    """Call Llama via Groq"""
    try:
        resp = groq_client.chat.completions.create(
            model=LLAMA_MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=10,
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

def call_deepseek(prompt: str) -> str:
    """Call DeepSeek"""
    try:
        resp = deepseek_client.chat.completions.create(
            model=DEEPSEEK_MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=10,
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"




In [4]:
def evaluate_third_party_tools(document_text):
    """Evaluate third-party tool SBOM and least-privilege"""
    system_prompt = (
        "You are a strict security and risk assessment expert. "
        "Your ONLY job is to read the document and output a single numeric score between 0 and 1. "
        "Do NOT explain, do NOT repeat the prompt, do NOT add any text."
    )
    
    user_prompt = f"""
Evaluate whether the following document demonstrates sufficient third-party tool SBOM and least-privilege validation.

Criteria:
1. SBOM completeness for tools/plugins/connectors
2. Permission-to-use mapping (scopes vs actual API calls)
3. Excess privilege rate (granted vs required permissions)
4. Secret/key isolation and rotation coverage for connectors
5. Data exfiltration path analysis (sink/source mapping)
6. Periodic re-validation rate (reviews/audits per connector)

Scoring:
- 0 = no mention of third-party tool tracking or permission validation
- 0.5 = mentions some tool tracking but lacks privilege analysis
- 1 = comprehensive SBOM with full permission validation and least-privilege enforcement
- Output a SINGLE number between 0 and 1 (inclusive).
- Do NOT output anything except the number.

Document:
{document_text}
"""
    
    full_prompt = system_prompt + "\n\n" + user_prompt
    
    print("\n" + "="*70)
    print("L4-08: Third-party tool/connector SBOM & least-privilege")
    print("="*70)
    
    deepseek_score = call_deepseek(full_prompt)
    print(f"DeepSeek Score: {deepseek_score}")
    
    llama_score = call_llama(full_prompt)
    print(f"Llama Score: {llama_score}")
    
    return {"deepseek": deepseek_score, "llama": llama_score}



In [7]:
def main():
    base_dir = Path.cwd().parent          
    docs_dir = base_dir / "docs" 
    doc_files = list(docs_dir.glob("sbom*.json"))

    if not doc_files:
        print("No SBOM documents found in docs/ folder")
        print(f"Searched in: {docs_dir.absolute()}")
        return
    
    results = {}
    
    for doc_file in doc_files:
        print(f"\n{'='*70}")
        print(f"Processing: {doc_file.name}")
        print(f"{'='*70}")
        
        # Read document
        with open(doc_file, 'r', encoding='utf-8') as f:
            document_text = f.read()
        
        # Truncate if too long (keep first 2000 chars)
        if len(document_text) > 2000:
            document_text = document_text[:2000] + "...[truncated]"
        
        doc_results = {}
        
        # Run all L4 evaluations
        doc_results['L4-08_Third_Party_Tools'] = evaluate_third_party_tools(document_text)
        
        results[doc_file.name] = doc_results
    
    # Save results to JSON
    output_file = Path("../l4_evaluation_results.json")
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n{'='*70}")
    print(f"Results saved to: {output_file}")
    print(f"{'='*70}")


if __name__ == "__main__":
    main()



Processing: sbom_deepseek_llm_env.json

L4-08: Third-party tool/connector SBOM & least-privilege
DeepSeek Score: 0.5
Llama Score: 0.5

Processing: sbom_llama_env.json

L4-08: Third-party tool/connector SBOM & least-privilege
DeepSeek Score: 0.5
Llama Score: 0.5

Results saved to: ../l4_evaluation_results.json
